In [ ]:
###############    V1    ################

import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# 1. Load Data
with open('intents.json', 'r') as f:
    data = json.load(f)

sentences = []
labels = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        sentences.append(pattern.lower())
        labels.append(intent['tag'])

# 2. Tokenization & Metadata
# We need to save the 'vocab' so your React Native app knows which number belongs to which word
vocab_size = 1000
embedding_dim = 16
max_len = 20
trunc_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating=trunc_type)

# 3. Label Encoding
lbl_encoder = LabelEncoder()
lbl_encoder.fit(labels)
training_labels = lbl_encoder.transform(labels)
num_classes = len(lbl_encoder.classes_)

# 4. Build the Model (Keras)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, np.array(training_labels), epochs=100)

# 5. Export to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('med_intent-1.tflite', 'wb') as f:
    f.write(tflite_model)

# 6. SAVE THESE FOR REACT NATIVE
# Save the word index (Vocabulary)
with open('vocab-1.json', 'w') as f:
    json.dump(word_index, f)

# Save the labels list
with open('labels-1.txt', 'w') as f:
    for label in lbl_encoder.classes_:
        f.write(label + '\n')

print("Files generated: med_intent-1.tflite, vocab-1.json, labels-1.txt")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.0164 - loss: 3.9515
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0408 - loss: 3.9485
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0300 - loss: 3.9453
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0318 - loss: 3.9420
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0453 - loss: 3.9388
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0360 - loss: 3.9276
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0316 - loss: 3.9209
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0357 - loss: 3.9207
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0263 - loss: 3.9097
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0344 - loss: 3.8939
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0397 - loss: 3.8664
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy

In [ ]:
###############    V2    ################

import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# 1. Load Data
# Ensure you have uploaded 'intents.json' to Colab first!
with open('intents.json', 'r') as f:
    data = json.load(f)

sentences = []
labels = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        sentences.append(pattern.lower())
        labels.append(intent['tag'])

# 2. Tokenization & Metadata (Hyperparameters optimized for CNN)
vocab_size = 1000
embedding_dim = 32 # Increased for better feature representation
max_len = 20
trunc_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating=trunc_type)

# 3. Label Encoding
lbl_encoder = LabelEncoder()
lbl_encoder.fit(labels)
training_labels = lbl_encoder.transform(labels)
num_classes = len(lbl_encoder.classes_)

# 4. Build the "Pro" 1D-CNN Model
model = tf.keras.Sequential([
    # Embedding: Turns words into dense vectors
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),

    # 1D-CNN: Captures relationships between neighboring words (n-grams)
    tf.keras.layers.Conv1D(64, 5, activation='relu'),

    # Global Max Pooling: Extracts the most important "emergency signal" from the sentence
    tf.keras.layers.GlobalMaxPooling1D(),

    # Fully Connected Layers
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2), # Dropout layer for better generalization (Resume point!)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train for 100 epochs (CNNs learn fast, but need enough epochs to converge)
model.fit(padded_sequences, np.array(training_labels), epochs=100)

# 5. Export to TFLite (Optimized for Mobile)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Optional: converter.optimizations = [tf.lite.Optimize.DEFAULT] # Adds Quantization (Smaller model)
tflite_model = converter.convert()

with open('med_intent-2.tflite', 'wb') as f:
    f.write(tflite_model)

# 6. SAVE THESE FOR YOUR TEAMMATES
# vocab.json: For the "Tokenizer Bridge" task
with open('vocab-2.json', 'w') as f:
    json.dump(word_index, f)

# labels.txt: The map for the UI to display the right name
with open('labels-2.txt', 'w') as f:
    for label in lbl_encoder.classes_:
        f.write(label + '\n')

print("\n--- ALL DONE ---")
print("Download these from the sidebar: med_intent-2.tflite, vocab-2.json, labels-2.txt")

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - accuracy: 0.0252 - loss: 3.9513
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0509 - loss: 3.9361
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0752 - loss: 3.9109
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1037 - loss: 3.8472
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1433 - loss: 3.6701
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2191 - loss: 3.3682
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2487 - loss: 2.9910
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3438 - loss: 2.5066
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4574 - loss: 2.1941
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5412 - loss: 1.8829
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6177 - loss: 1.5605
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6788 - l

In [ ]:
###############    V3    ################

import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# 1. Preprocessing Function (Use the EXACT same one in React Native!)
STOP_WORDS = ["help", "i", "think", "got", "my", "me", "the", "a", "an", "please", "somebody", "do", "does"]

def clean_text(text):
    text = text.lower()
    # Remove basic punctuation
    for char in '?!.,':
        text = text.replace(char, '')
    # Remove stop words
    words = text.split()
    filtered = [w for w in words if w not in STOP_WORDS]
    return " ".join(filtered)

# 2. Load and Clean Data
with open('intents.json', 'r') as f:
    data = json.load(f)

sentences = []
labels = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        cleaned_pattern = clean_text(pattern)
        if cleaned_pattern: # Ensure it's not empty after cleaning
            sentences.append(cleaned_pattern)
            labels.append(intent['tag'])

# 3. Tokenization (Using the cleaned sentences)
vocab_size = 1000
max_len = 15 # Reduced since sentences are shorter now
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# 4. Label Encoding
lbl_encoder = LabelEncoder()
training_labels = lbl_encoder.fit_transform(labels)
num_classes = len(lbl_encoder.classes_)

# 5. Build CNN Model (The "Pro" configuration from before)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=max_len),
    tf.keras.layers.Conv1D(32, 3, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, np.array(training_labels), epochs=150, verbose=0)

# 6. Export Files
# (Same export code as before for .tflite, vocab.json, and labels.txt)

# 5. Export to TFLite (Optimized for Mobile)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Optional: converter.optimizations = [tf.lite.Optimize.DEFAULT] # Adds Quantization (Smaller model)
tflite_model = converter.convert()

with open('med_intent-3.tflite', 'wb') as f:
    f.write(tflite_model)

# 6. SAVE THESE FOR YOUR TEAMMATES
# vocab.json: For the "Tokenizer Bridge" task
with open('vocab-3.json', 'w') as f:
    json.dump(word_index, f)

# labels.txt: The map for the UI to display the right name
with open('labels-3.txt', 'w') as f:
    for label in lbl_encoder.classes_:
        f.write(label + '\n')

print("\n--- ALL DONE ---")
print("Download these from the sidebar: med_intent-3.tflite, vocab-3.json, labels-3.txt")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Saved artifact at '/tmp/tmp7iwipb47'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 15), dtype=tf.float32, name='keras_tensor_41')
Output Type:
  TensorSpec(shape=(None, 52), dtype=tf.float32, name=None)
Captures:
  135207619416464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135207619415504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135207619415120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135207619412624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135207619415696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135207619414736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135207619416848: TensorSpec(shape=(), dtype=tf.resource, name=None)

--- ALL DONE ---
Download these from the sidebar: med_intent.tflite, vocab.json, labels.txt


In [ ]:
###############    V4    ################

import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# 1. Body Part Augmentation List
body_parts = ["ribs", "chest", "skull", "wrist", "ankle", "hip", "spine", "nose", "toe", "shoulder", "jaw"]
fracture_synonyms = ["fractured", "broken", "cracked", "snapped"]

# 2. Preprocessing Function
STOP_WORDS = ["help", "i", "think", "got", "my", "me", "the", "a", "an", "please", "somebody"]

def clean_text(text):
    text = text.lower()
    for char in '?!.,': text = text.replace(char, '')
    return " ".join([w for w in text.split() if w not in STOP_WORDS])

# 3. Load and Augment Data
with open('intents.json', 'r') as f:
    data = json.load(f)

sentences = []
labels = []

for intent in data['intents']:
    tag = intent['tag']

    # 1. Add original patterns
    for pattern in intent['patterns']:
        cleaned = clean_text(pattern)
        if cleaned:
            sentences.append(cleaned)
            labels.append(tag)

    # 2. Add Systematic Augmentation for fractures
    if tag == 'fracture':
        for part in body_parts:
            for syn in fracture_synonyms:
                # OPTION A: "ribs fractured"
                phrase1 = clean_text(f"{part} {syn}")
                if phrase1:
                    sentences.append(phrase1)
                    labels.append(tag) # Correctly paired

                # OPTION B: "i think i broke my ribs"
                phrase2 = clean_text(f"i think i {syn} my {part}")
                if phrase2:
                    sentences.append(phrase2)
                    labels.append(tag) # Correctly paired

# Final sanity check print
print(f"Total Sentences (X): {len(sentences)}")
print(f"Total Labels (Y): {len(labels)}")
# These two numbers MUST be identical now.

# 4. Tokenization & Model
vocab_size = 1000
max_len = 15
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

lbl_encoder = LabelEncoder()
training_labels = lbl_encoder.fit_transform(labels)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32, input_length=max_len),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    # Use Max Pooling to catch the specific 'fracture' signal
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(len(lbl_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, np.array(training_labels), epochs=120, verbose=1)


# 6. Export Files
# (Same export code as before for .tflite, vocab.json, and labels.txt)

# 5. Export to TFLite (Optimized for Mobile)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Optional: converter.optimizations = [tf.lite.Optimize.DEFAULT] # Adds Quantization (Smaller model)
tflite_model = converter.convert()

with open('med_intent-4.tflite', 'wb') as f:
    f.write(tflite_model)

# 6. SAVE THESE FOR YOUR TEAMMATES
# vocab.json: For the "Tokenizer Bridge" task
with open('vocab-4.json', 'w') as f:
    json.dump(word_index, f)

# labels.txt: The map for the UI to display the right name
with open('labels-4.txt', 'w') as f:
    for label in lbl_encoder.classes_:
        f.write(label + '\n')

print("\n--- ALL DONE ---")
print("Download these from the sidebar: med_intent-4.tflite, vocab-4.json, labels-4.txt")

Total Sentences (X): 1081
Total Labels (Y): 1081
Epoch 1/120


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - accuracy: 0.0962 - loss: 3.9351
Epoch 2/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1811 - loss: 3.8245
Epoch 3/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1690 - loss: 3.6297
Epoch 4/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1751 - loss: 3.3979
Epoch 5/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2069 - loss: 3.2174
Epoch 6/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2117 - loss: 3.1410
Epoch 7/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2617 - loss: 2.8813
Epoch 8/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2961 - loss: 2.7456
Epoch 9/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3401 - loss: 2.5702
Epoch 10/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3742 - loss: 2.3106
Epoch 11/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4183 - loss: 2.0986
Epoch 12/120
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5026 - l

In [ ]:
def predict_intent(text):
    # 1. Pre-process the input string
    sequence = tokenizer.texts_to_sequences([text.lower()])
    padded = pad_sequences(sequence, maxlen=max_len, truncating=trunc_type)

    # 2. Run prediction
    prediction = model.predict(padded)

    # 3. Get the label with the highest probability
    class_idx = np.argmax(prediction)
    tag = lbl_encoder.inverse_transform([class_idx])[0]
    confidence = prediction[0][class_idx]

    return tag, confidence

# --- TEST IT HERE ---
test_sentences = [
    "I'm bleeding really bad from a knife cut",
    "My friend passed out and isn't waking up",
    "I have a sharp pain in my chest",
    "Help I think i got my arms fractured"
]

for sentence in test_sentences:
    intent, prob = predict_intent(sentence)
    print(f"Text: {sentence}")
    print(f"AI Prediction: {intent} ({prob*100:.2f}% confidence)\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
Text: I'm bleeding really bad from a knife cut
AI Prediction: severe_bleeding (78.84% confidence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Text: My friend passed out and isn't waking up
AI Prediction: unconscious_unresponsive (65.05% confidence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Text: I have a sharp pain in my chest
AI Prediction: severe_bleeding (61.95% confidence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Text: Help I think i got my arms fractured
AI Prediction: head_injury (19.77% confidence)



In [ ]:
import numpy as np

# 1. The Preprocessing Function (Must be identical to training)
STOP_WORDS = ["help", "i", "think", "got", "my", "me", "the", "a", "an", "please", "somebody", "do", "does"]

def clean_text_test(text):
    text = text.lower()
    for char in '?!.,':
        text = text.replace(char, '')
    words = text.split()
    filtered = [w for w in words if w not in STOP_WORDS]
    return " ".join(filtered)

# 2. The Prediction Function
def predict_emergency(raw_text):
    # Step 1: Clean
    cleaned = clean_text_test(raw_text)

    # Step 2: Tokenize and Pad
    sequence = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post')

    # Step 3: Inference
    pred = model.predict(padded, verbose=0)

    # Step 4: Map back to Label
    class_idx = np.argmax(pred)
    tag = lbl_encoder.inverse_transform([class_idx])[0]
    confidence = pred[0][class_idx]

    return cleaned, tag, confidence

# 3. RUN THE STRESS TEST
test_cases = [
    "Help I think i got my ribs fractured", # Your problematic case
    "Someone please help me, my chest hurts",
    "I have a minor cut on my finger",
    "Help somebody is having a seizure",
    "I think I swallowed some poison",
    "My friend passed out and isn't waking up"
]

print(f"{'Original Text':<40} | {'Cleaned':<20} | {'Prediction':<15} | {'Conf.'}")
print("-" * 95)

for text in test_cases:
    cleaned, tag, conf = predict_emergency(text)
    print(f"{text:<40} | {cleaned:<20} | {tag:<15} | {conf:.2%}")

Original Text                            | Cleaned              | Prediction      | Conf.
-----------------------------------------------------------------------------------------------
Help I think i got my ribs fractured     | ribs fractured       | fracture        | 99.99%
Someone please help me, my chest hurts   | someone chest hurts  | chest_pain      | 96.41%
I have a minor cut on my finger          | have minor cut on finger | cuts            | 99.96%
Help somebody is having a seizure        | is having seizure    | seizure         | 98.72%
I think I swallowed some poison          | swallowed some poison | poisoning       | 97.15%
My friend passed out and isn't waking up | friend passed out and isn't waking up | unconscious_unresponsive | 90.61%


In [ ]:
def final_stress_test():
    test_cases = [
        "Help I think i got my ribs fractured",
        "My wrist is broken help",
        "I think I'm drowning",
        "i have a fracture in my leg"
    ]

    print(f"{'Input':<40} | {'Prediction'}")
    print("-" * 60)
    for test in test_cases:
        _, tag, conf = predict_emergency(test) # Using the function from earlier
        print(f"{test:<40} | {tag} ({conf:.2%})")

final_stress_test()

Input                                    | Prediction
------------------------------------------------------------
Help I think i got my ribs fractured     | fracture (99.99%)
My wrist is broken help                  | fracture (99.61%)
I think I'm drowning                     | drowning (99.83%)
i have a fracture in my leg              | fracture (79.61%)


In [ ]:
test_phrase = "Help I think i got my ribs fractured"
cleaned = clean_text(test_phrase)
seq = tokenizer.texts_to_sequences([cleaned])
padded = pad_sequences(seq, maxlen=max_len, padding='post')
pred = model.predict(padded, verbose=0)
tag = lbl_encoder.inverse_transform([np.argmax(pred)])[0]

print(f"Input: {test_phrase}")
print(f"Cleaned: {cleaned}")
print(f"Result: {tag} ({np.max(pred):.2%})")

Input: Help I think i got my ribs fractured
Cleaned: ribs fractured
Result: fracture (100.00%)


In [ ]:
import json

def update_intents_json(file_path):
    # 1. Load the existing file
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2. Define our augmentation data (from your previous logic)
    body_parts = ["ribs", "chest", "skull", "wrist", "ankle", "hip", "spine", "nose", "toe", "shoulder", "jaw"]
    fracture_synonyms = ["fractured", "broken", "cracked", "snapped"]

    # 3. Find the fracture intent and update it
    for intent in data['intents']:
        if intent['tag'] == 'fracture':
            # Get existing patterns to avoid duplicates
            existing_patterns = set(intent['patterns'])

            new_count = 0
            for part in body_parts:
                for syn in fracture_synonyms:
                    # Create two common variations
                    p1 = f"{part} {syn}"
                    p2 = f"I think I {syn} my {part}"

                    if p1 not in existing_patterns:
                        intent['patterns'].append(p1)
                        existing_patterns.add(p1)
                        new_count += 1

                    if p2 not in existing_patterns:
                        intent['patterns'].append(p2)
                        existing_patterns.add(p2)
                        new_count += 1

            print(f"Added {new_count} new patterns to the 'fracture' tag.")

    # 4. Save the final JSON back to the file
    # We use indent=4 to keep it readable for your teammates
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

    print(f"Successfully updated {file_path}")

# Execute
update_intents_json('intents.json')

Added 88 new patterns to the 'fracture' tag.
Successfully updated intents.json
